In [1]:
import numpy as np
import pandas as pd

In [149]:
def read_csv(date, table_name):
    year, month, day = date
    path = f'{year}{month:02d}{day:02d}/{table_name}.csv'
    return pd.read_csv(path, nrows=None, sep=',', on_bad_lines='skip', encoding='utf-8')

In [58]:
# date = (2023, 2, 1)
# chain = read_csv(date, 'KSCC_DX_TRIP_CHAIN_SUM')
# station = read_csv(date, 'TBIS_MS_STTN')
# district = read_csv(date, 'DISTRICT_EMD')

In [161]:
station1 = pd.read_excel('6월 기준 행정권역 정류장정보.xlsx')
station1 = station1.dropna()
station1 = station1.loc[~station1['정류장번호'].str.contains('-')]
station1 = station1.astype({'정류장번호':'int'})
station1 = station1.loc[station1['정류장번호'] != 0]

station2 = pd.read_excel('20220429기준_서울시정류소리스트.xlsx')

In [180]:
station = pd.merge(station1, station2, left_on='정류장번호', right_on='ARS-ID')
station = station.drop(['NO', '정류장명칭', '정류장번호'], axis=1)
station = station.rename(columns={'NODE_ID':'STTN_ID', '시군구명':'행정구', '행정동명':'행정동', 'ARS-ID':'ARS_ID', '좌표X':'경도', '좌표Y':'위도'})
station = station[['STTN_ID', 'ARS_ID', '정류소명', '행정구', '행정동', '경도', '위도']]
station['ARS_ID'] = station['ARS_ID'].map(lambda x: f'{x:05d}')

In [182]:
station.to_csv('bus-station-master.csv', index=False)

In [185]:
station

,STTN_ID,ARS_ID,정류소명,행정구,행정동,경도,위도
0,100000379,01008,서울역사박물관.경희궁앞,종로구,사직동,126.970442,37.569336
1,100000380,01007,서울역사박물관.경희궁앞,종로구,사직동,126.970420,37.569098
2,100000384,01009,광화문,종로구,사직동,126.975241,37.570051
3,100000018,01113,사직단.어린이도서관,종로구,사직동,126.968021,37.575113
4,100000019,01114,사직동주민센터,종로구,사직동,126.971318,37.576090
...,...,...,...,...,...,...,...
10360,117000906,18129,독산주공14단지,광명시,하안3동,126.887041,37.461346
10361,117000907,18128,독산주공14단지,광명시,하안3동,126.886996,37.461206
10362,117900165,18993,안양천로입구,광명시,소하1동,126.893035,37.452452
10363,124000369,25003,초이동,하남시,초이동,127.161382,37.541971


In [188]:
tibs_station = read_csv((2023, 2, 1), 'TBIS_MS_STTN')

In [189]:
tibs_station = tibs_station.loc[tibs_station['STTN_NO'].notnull()]
tibs_station['STTN_NO'] = tibs_station['STTN_NO'].astype('int')
tibs_station = tibs_station.loc[tibs_station['STTN_NO'] != 0]
tibs_station['STTN_NO'] = tibs_station['STTN_NO'].map(lambda x: f'{x:05d}')

In [190]:
tibs_station

,STTN_ID,STTN_NM,STTN_TY,STTN_NO,CRDNT_X,CRDNT_Y,BIT_INSTL_AT
0,216000387,안산갈대습지,일반차로,17385,126.834835,37.279582,미설치
1,163000582,시립가좌테니스장,일반차로,37582,126.678192,37.471517,미설치
2,163000581,시립가좌테니스장,일반차로,37581,126.678401,37.471847,미설치
3,163000580,인천기계산업단지,일반차로,37580,126.670796,37.469205,미설치
4,218000350,북한산유보라아파트,일반차로,57547,126.912444,37.650976,미설치
...,...,...,...,...,...,...,...
61395,229000652,LCD산업단지,일반차로,30873,126.767169,37.812516,미설치
61396,212000374,오정경찰서.부천옹기박물관,일반차로,45641,126.801082,37.513832,미설치
61398,228000599,라이프아파트,일반차로,29458,127.213939,37.231994,미설치
61399,233000645,신한미지엔아파트앞,일반차로,36304,127.008633,37.199851,미설치


In [192]:
# master = pd.merge(tibs_station, df, left_on='STTN_NO', right_on='ARS_ID', how='inner')
master = pd.merge(tibs_station, station, on='STTN_ID', how='inner')

master = master.drop(['STTN_NO', 'STTN_NM'], axis=1)
master = master.rename(columns={'STTN_TY':'버스유형', 'CRDNT_X':'경도2', 'CRDNT_Y':'위도2', 'BIT_INSTL_AT':'버스도착정보안내기'})
master = master[['STTN_ID', 'ARS_ID', '행정구', '행정동', '정류소명', '경도', '위도', '경도2', '위도2', '버스유형', '버스도착정보안내기']]
master = master.loc[~master['버스유형'].isin(['마을버스', '가상정류장'])]

In [194]:
master

,STTN_ID,ARS_ID,행정구,행정동,정류소명,경도,위도,경도2,위도2,버스유형,버스도착정보안내기
8,118000153,19238,영등포구,신길4동,우신초등학교,126.910840,37.509233,126.910940,37.509252,일반차로,미설치
9,118000154,19239,영등포구,신길4동,신길힐스테이트클래시안아파트,126.910672,37.505036,126.910811,37.504989,일반차로,설치
10,118000155,19240,영등포구,신길4동,신길동홍어거리,126.911546,37.510873,126.911572,37.510794,일반차로,미설치
11,118000156,19241,영등포구,신길5동,신풍역.백악관웨딩문화원앞,126.909456,37.500893,126.909880,37.501582,일반차로,설치
12,118000157,19242,영등포구,신길5동,신길5동주민센터,126.905490,37.500633,126.905541,37.500654,일반차로,미설치
...,...,...,...,...,...,...,...,...,...,...,...
10358,110000244,11345,노원구,월계1동,인덕삼거리,127.057136,37.628587,127.057162,37.628699,일반차로,설치
10359,110000245,11346,노원구,월계1동,신도브래뉴.한전노원변전소,127.059169,37.628234,127.058713,37.628526,일반차로,설치
10360,110000246,11347,노원구,중계2·3동,상명고등학교.성원아파트,127.069183,37.649747,127.069232,37.649776,일반차로,미설치
10361,110000247,11348,노원구,중계1동,중계1동주민센터,127.077277,37.651968,127.077233,37.651993,일반차로,설치


In [ ]:
# master['행정구'].value_counts()
# station['행정구'].value_counts()

In [198]:
subway = read_csv((2023, 2, 1), 'KSCC_DV_SUBWAY_STATION_GEOM')

In [199]:
subway

,OUT_STN_NUM,STN_KR_NM,LINE_NM,CONV_X,CONV_Y
0,2563,미사,5호선,127.193877,37.560927
1,2564,하남풍산,5호선,127.203864,37.552034
2,3139,송도달빛축제공원,인천1호선,126.625970,37.407143
3,4203,홍대입구,공항철도1호선,126.926715,37.557438
4,428,삼각지,4호선,126.972600,37.534075
...,...,...,...,...,...
760,3760,굴포천,7호선(인천),126.732340,37.506810
761,405,진접,진접선,127.203400,37.720500
762,3761,부평구청,7호선(인천),126.720770,37.508480
763,406,오남,진접선,127.192810,37.705000
